In [1]:
import numpy as np
import pandas as pd
import copy
import csv

from Bio import SeqIO, SearchIO 

In [3]:
file = '/home/marfa/Desktop/lab/mycofactocin/mftb/hmm_for_domains/parsed_hmm_per_dom_tbl.txt'
res_domains = dict()
x = 10
count = 0
for res in SearchIO.parse(file, 'hmmscan3-domtab'):
        res_domains[res.id] = {}
        for hit in res.hits:
            if hit.id[:4] == 'TIGR':
                    call = hit.description.split(":")[0]
            else:
                    call = hit.id
            for hsp in hit.hsps:
                overlap = False
                if res_domains[res.id].keys(): #not empty
                    names = copy.deepcopy(list(res_domains[res.id].keys()))
                    for name in names: #проверка на перекрытие с раннее добавленными генами 
                        for q_range in res_domains[res.id][name]:
                            if min(hsp.env_end, q_range[1]) - max(hsp.env_start, q_range[0]) > x:
                                overlap = True
                                break
                        if overlap:
                            break
                    if not overlap: 
                        if call not in res_domains[res.id]:
                            res_domains[res.id][call] = [(hsp.env_start, hsp.env_end)]
                            
                        if (hsp.env_start, hsp.env_end) not in res_domains[res.id][call]:
                                        #для случая мультидоменного белка с одинаковыми доменами: 
                                        #один и тот же белок выравнивается на несколько мест
                            res_domains[res.id][call].append((hsp.env_start, hsp.env_end))
                       
                else:
                    res_domains[res.id][call] = [(hsp.env_start, hsp.env_end)]

In [4]:
for ke, value in res_domains.items(): #сортирую по началу выравнивания гена на регион
    res_domains[ke] = {k: v for k, v in sorted(res_domains[ke].items(), key=lambda item: item[1][0][0])}

res_domains

{'WP_011566059.1': {'MnhB': [(9, 136)]},
 'WP_011566060.1': {'2a6301s01': [(27, 758)]},
 'WP_011566061.1': {'Methyltransf_23': [(35, 79)],
  'Methyltransf_12': [(115, 207)],
  'mycofact_glyco': [(212, 253), (260, 735)]},
 'WP_011566062.1': {'Creatininase': [(5, 244)]},
 'WP_011566063.1': {'actino_HemFlav': [(3, 390)]},
 'WP_011566064.1': {'AAA_5': [(32, 171)],
  'ccmA': [(214, 244)],
  'AAA_16': [(343, 484)],
  'bpMoxR': [(499, 522)],
  'marine_srt_targ': [(569, 619)]},
 'WP_011566065.1': {'Rxyl_3153': [(1, 371)]},
 'WP_011566066.1': {'mycofact_rSAM': [(2, 339)]},
 'WP_011566067.1': {'mycofact_MftB': [(5, 89)]},
 'WP_011566068.1': {'SDR_subfam_1': [(9, 283)]},
 'WP_083760118.1': {'mycofactocin': [(34, 57)]},
 'WP_011566069.1': {'KCNQ2_u3': [(14, 35)], 'N6_N4_Mtase': [(32, 247)]},
 'WP_198004859.1': {'panF': [(0, 472)]},
 'WP_041329379.1': {'DUF3311': [(3, 60)]},
 'WP_198004860.1': {'4HBT_2': [(5, 138)]},
 'WP_011566073.1': {'TPR_2': [(2, 36),
   (36, 70),
   (74, 104),
   (104, 138),
 

In [19]:
df = {}
with open('/home/marfa/Desktop/lab/mycofactocin/mftb/flags/mftb_g8_flankgene.fasta_cluster_out_3_1e-10_outdesc.txt') as file:
    for i, line in enumerate(file):
        line = line.rstrip('\n')
        line = line.split("\t")
        if len(line)>1:
            prot_no = line[0].split('(')[0]
            prot_id = line[1]
            if prot_id in res_domains:
                domain = list(res_domains[prot_id].keys())
            else:
                domain = np.nan
            name = ''
            if type(domain) != float: #not NaN
                for dom in domain: 
                    if dom == 'mycofactocin':
                        name += ' MftA'
                    elif dom == 'mycofact_TetR' or name == 'TetR_N':
                        name += ' MftR'
                    elif dom == 'mycofact_rSAM':
                        name += ' MftC'
                    elif dom == 'Rv0694' or dom == 'actino_HemFlav':
                        name += ' MftD'
                    elif dom == 'mycofact_glyco':
                        name += ' MftF'
                    elif dom == 'mft_etfB':
                        name += ' mycofact_flavoprotein'
            df[prot_id] = [prot_no, domain, name]


df = pd.DataFrame({'prot_no':[x[0] for x in df.values()], 'domain': [x[1] for x in df.values()], 'name': [x[2] for x in df.values()]}, index = df.keys())
df

,prot_no,domain,name
WP_235545783.1,1,[SDR_subfam_2],
WP_067395275.1,1,[Ac_CoA_lig_AcsA],
WP_179483125.1,1,[quinone_pig3],
WP_031469035.1,1,[SDR_subfam_2],
WP_212844273.1,1,"[AA-adenyl-dom, alpha_am_amid, Condensation]",
...,...,...,...
WP_257909995.1,415,"[F420_oxidored, tartro_sem_red]",
WP_260430896.1,416,NaN,
WP_041255986.1,416,[lipA],
WP_215526093.1,417,[MtN3_slv],


In [20]:
#df.to_csv('/home/marfa/Desktop/lab/mycofactocin/mftb/flags/mftb_g8_flankgene_desc.csv')